In [1]:
import pandas as pd
import numpy as np

import os

def load_hotel_reserve():
  customer_tb = pd.read_csv('./data/customer.csv')
  hotel_tb = pd.read_csv('./data/hotel.csv')
  reserve_tb = pd.read_csv('./data/reserve.csv')
  return customer_tb, hotel_tb, reserve_tb


def load_holiday_mst():
  holiday_tb = pd.read_csv('./data/holiday_mst.csv',
                           index_col=False)
  return holiday_tb


def load_production():
  production_tb = pd.read_csv('./data/production.csv')
  return production_tb


def load_production_missing_num():
  production_tb = pd.read_csv('./data/production_missing_num.csv')
  return production_tb


def load_production_missing_category():
  production_tb = pd.read_csv('./data/production_missing_category.csv')
  return production_tb


def load_monthly_index():
  monthly_index_tb = \
    pd.read_csv('./data/monthly_index.csv')
  return monthly_index_tb


def load_meros_txt():
  with open('./data/txt/meros.txt', 'r') as f:
    meros = f.read()
    f.close()
  return meros


In [2]:
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()

### 3회차 – 데이터의 세부 내용 전처리

#### 1. 수치 데이터 전처리 기법
- 수치 데이터 형식으로의 변환 

    -> 원칙적으로 수치형 데이터열은 수치형으로 자동 변환

    -> 값에 문자가 섞이거나 정수형/부동소수점형의 변환이 필요한 경우 전처리 작업 필요

In [15]:
# 다양한 데이터형 변환
df = pd.DataFrame({"value": [40000 / 3]})

print(df.dtypes)

print('\n') # 다양한 사이즈의 정수형 변환
print(df['value'].astype('int8'))
print(df['value'].astype('int16'))
print(df['value'].astype('int32'))
print(df['value'].astype('int64'))

print('\n') # 다양한 사이즈의 실수형 변환
print(df['value'].astype('float16'))
print(df['value'].astype('float32'))
print(df['value'].astype('float64'))

print('\n') # 데이터형 지정 가능
print(df['value'].astype('int'))
print(df['value'].astype('float'))

value    float64
dtype: object


0    21
Name: value, dtype: int8
0    13333
Name: value, dtype: int16
0    13333
Name: value, dtype: int32
0    13333
Name: value, dtype: int64


0    13336.0
Name: value, dtype: float16
0    13333.333008
Name: value, dtype: float32
0    13333.333333
Name: value, dtype: float64


0    13333
Name: value, dtype: int64
0    13333.333333
Name: value, dtype: float64


c:\Users\EL087\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


- 로그 변환 등 지수 기반 스케일링

    -> 선형 모델로 표현할 수 없는 문제를 비선형 모델로 다루기 위해 수치를 변환하는 전처리가 필요

    -> b = log𝑎 𝑥 식을 이용하여 b 값을 구하는 방법을 대수화라고 함

In [ ]:
# hotel_price를 대수화하여 전처리
reserve_tb['total_price_log'] = \
    reserve_tb['total_price'].apply(lambda x: np.log10(x / 1000 + 1))   # x가 0임을 피하기 위해 +1

reserve_tb['total_price_log']

0       1.992111
1       1.334454
2       1.539076
3       2.290925
4       1.839478
          ...   
4025    1.230449
4026    1.631444
4027    1.879669
4028    2.733197
4029    1.654177
Name: total_price_log, Length: 4030, dtype: float64

- 순위 또는 분위수를 활용한 스케일링

    -> 범주화란 수치를 다수의 플래그값(TRUE or FALSE)으로 변환하는 것

    -> 대수화만으로 복잡한 변화를 충분히 표현할 수 없을 때 범주화를 사용

    - 대수화: 데이터의 값이 연속적이거나 수학적인 크기를 가지는 경우

In [17]:
# 고객 테이블의 나이를 10단위의 범주형으로 추가
customer_tb['age_rank'] = \
    (np.floor(customer_tb['age'] / 10 ) * 10).astype('category')

print(customer_tb['age_rank'])

0      40.0
1      30.0
2      40.0
3      40.0
4      30.0
       ... 
995    40.0
996    30.0
997    30.0
998    40.0
999    30.0
Name: age_rank, Length: 1000, dtype: category
Categories (7, float64): [20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0]


- 정규화

    -> 값이 가질 수 있는 범위를 통일하는 변환처리

    -> 머신러닝 모델의 입력값이 가질 수 있는 범위를 균일하게 유지하는 역할

    -> 정규화의 두 가지 방법
    - 평균 0, 분산 1로 변환하는 정규화
    - 최소값 0, 최대값 1로 변환하는 정규화


In [18]:
# 예약 인수와 합계 금액을 평균 0, 분산 1의 분포로 변환하여 정규화
from sklearn.preprocessing import StandardScaler

reserve_tb['people_num'] = reserve_tb['people_num'].astype(float)

ss = StandardScaler()

result = ss.fit_transform(reserve_tb[['people_num', 'total_price']])

reserve_tb['people_num_normalized'] = [x[0] for x in result]
reserve_tb['total_price_normalized'] = [x[1] for x in result]

reserve_tb['people_num_normalized']

0       1.300709
1      -0.483753
2      -0.483753
3       1.300709
4       0.408478
          ...   
4025   -0.483753
4026   -0.483753
4027   -0.483753
4028    1.300709
4029   -1.375984
Name: people_num_normalized, Length: 4030, dtype: float64

- 이상값 처리

    -> 대부분 값보다 극단적으로 크거나 작은 값, 즉 예외 값의 영향으로 문제가 발생

    -> 이는 정규화나 예측모델 구축 시 나쁜 영향을 끼치므로 전처리 단계에서 제거

    -> 정규분포를 전제로 한 예외 값 검출방법
    - 표준편차의 일정 배수 이상 떨어진 값을 평균값에서 제거하는 방법
    - 정규분포에 따른 값은 평균값에서 표준편차의 세 배 이내 범위에 99.73%가 있으므로 0.27%를 예외 값으로 봄

In [20]:
# 예약 테이블의 예약 합계금액의 평균값에서 표준편차의 세 배 이외의 값을 삭제
reserve_tb = reserve_tb[
    (abs(reserve_tb['total_price'] - np.mean(reserve_tb['total_price'])) / 
     np.std(reserve_tb['total_price']) <= 3)
].reset_index()

reserve_tb

,level_0,index,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,total_price_log,people_num_normalized,total_price_normalized
0,0,0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4.0,97200,1.992111,1.300709,-0.053194
1,1,1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2.0,20600,1.334454,-0.483753,-0.747822
2,2,2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2.0,33600,1.539076,-0.483753,-0.629935
3,3,3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4.0,194400,2.290925,1.300709,0.828240
4,4,4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3.0,68100,1.839478,0.408478,-0.317080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840,3927,4024,r4025,h_160,c_999,2017-03-11 11:56:05,2017-03-27,10:00:00,2017-03-30,1.0,37200,1.582063,-1.375984,-0.597289
3841,3928,4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2.0,16000,1.230449,-0.483753,-0.789536
3842,3929,4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2.0,41800,1.631444,-0.483753,-0.555575
3843,3930,4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2.0,74800,1.879669,-0.483753,-0.256323


- 주성분 기반 차원 축소

    -> 입력값의 종류가 많을수록 머신러닝 모델이 경향을 학습하는데 필요한 데이터의 양이 커짐

    -> 주성분 분석을 이용하여 차원(입력값의 종류)을 줄이면 데이터 양이 줄여 쉽게 학습 가능

In [21]:
# PCA 클래스를 이용하여 주성분 분석을 구현
production_tb = load_production()

from sklearn.decomposition import PCA

pca = PCA(n_components=2)   # 2개의 주성분으로 PCA 객체 생성

pca_values = pca.fit_transform(production_tb[['length', 'thickness']])

print('누적 기여율: {0}'.format(sum(pca.explained_variance_ratio_)))
print('각 차원의 기여율: {0}'.format(pca.explained_variance_ratio_))

pca_newvalues = pca.transform(production_tb[['length', 'thickness']])

pca_newvalues

누적 기여율: 1.0
각 차원의 기여율: [0.97897794 0.02102206]


array([[  76.96838157,   13.38906936],
       [-112.11469337,    8.24884796],
       [ -76.1994339 ,  -11.19027127],
       ...,
       [  31.12100559,  -15.48152593],
       [-117.87675543,   -2.4361334 ],
       [   4.80243541,   15.32174872]], shape=(1000, 2))

- 결측값 처리 기법 - 누락된 행 제거

In [ ]:
# thickness에 결손이 발생한 제조 레코드 사용 시 결손된 레코드 제거
production_miss_num = load_production_missing_num()

production_miss_num.replace('None', np.nan, inplace=True) # None을 nan으로 교체

production_miss_num.dropna(subset=['thickness'], inplace=True) # dropna로 nan 레코드 제거

production_miss_num

,type,length,thickness,fault_flg
0,E,274.027383,40.241131,False
1,D,86.319269,16.906715,False
2,E,123.940388,1.018462,False
3,B,175.554886,16.414924,False
4,B,244.934740,29.061081,False
...,...,...,...,...
995,C,363.214163,48.369483,False
996,D,134.773797,26.861665,False
997,B,231.174985,7.087471,False
998,D,81.613510,5.716271,False


- 결측값 처리 기법 - 고정값 대체

In [25]:
# thickness에 결손이 발생한 제조 레코드 사용 시 1로 보완
production_miss_num.replace('None', np.nan, inplace=True) # None을 nan으로 교체

production_miss_num['thickness'].fillna(1, inplace=True) # nan 값을 1로 대체

production_miss_num

C:\Users\EL087\AppData\Local\Temp\ipykernel_9016\1009159008.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  production_miss_num['thickness'].fillna(1, inplace=True) # nan 값을 1로 대체


,type,length,thickness,fault_flg
0,E,274.027383,40.241131,False
1,D,86.319269,16.906715,False
2,E,123.940388,1.018462,False
3,B,175.554886,16.414924,False
4,B,244.934740,29.061081,False
...,...,...,...,...
995,C,363.214163,48.369483,False
996,D,134.773797,26.861665,False
997,B,231.174985,7.087471,False
998,D,81.613510,5.716271,False


- 결측값 처리 기법 - 평균값 대체

In [26]:
# thickness에 결손이 발생한 제조 레코드 사용 시 평균값으로 보완
production_miss_num.replace('None', np.nan, inplace=True) # None을 nan으로 교체

production_miss_num['thickness'] = \
    production_miss_num['thickness'].astype('float64') # 실수값으로 변경

thickness_mean = production_miss_num['thickness'].mean()

production_miss_num['thickness'].fillna(thickness_mean, inplace=True) # nan을 평균값으로 교체

production_miss_num

C:\Users\EL087\AppData\Local\Temp\ipykernel_9016\1105845556.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  production_miss_num['thickness'].fillna(thickness_mean, inplace=True) # nan을 평균값으로 교체


,type,length,thickness,fault_flg
0,E,274.027383,40.241131,False
1,D,86.319269,16.906715,False
2,E,123.940388,1.018462,False
3,B,175.554886,16.414924,False
4,B,244.934740,29.061081,False
...,...,...,...,...
995,C,363.214163,48.369483,False
996,D,134.773797,26.861665,False
997,B,231.174985,7.087471,False
998,D,81.613510,5.716271,False
